In [1]:
import pandas as pd
import pickle
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error
import mlflow
from sklearn.model_selection import train_test_split
import os
from mlflow.tracking import MlflowClient
import xgboost as xgb
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import LinearSVR

In [2]:
mlflow.set_tracking_uri("sqlite:///mlflow.db")
#mlflow.set_experiment("airbnb-price-experiment")
client = MlflowClient("http://127.0.0.1:5000")

In [16]:
categorical = ['neighbourhood', 'room_type']
numerical = ['availability_365']
target = ['price']

In [17]:
def read_dataframe(filename):
    df = pd.read_csv(filename, low_memory=False)
    df = df[categorical + numerical + target].dropna()
    df[categorical] = df[categorical].astype(str)
    df_train, df_remain = train_test_split(df, test_size=0.3)
    df_val, df_test = train_test_split(df_remain, test_size=0.3)

    return (df_train, df_val, df_test)

In [18]:
df_train, df_val, df_test = read_dataframe('./data/NYC-Airbnb-2023.csv')

In [19]:
len(df_train), len(df_val), len(df_test)

(22838, 6852, 2937)

In [35]:
dv = DictVectorizer()
train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

y_train = df_train[target]
y_val = df_val[target]

In [39]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

1231.696570359967

In [40]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

with open("models/preprocessor.b", "wb") as f_out:
    pickle.dump(dv, f_out)

In [41]:
with mlflow.start_run():

    mlflow.set_tag("developer", "cristian")

    mlflow.log_param("data-path", "./data/NYC-Airbnb-2023.csv")

    alpha = 0.1
    mlflow.log_param("alpha", alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    mlflow.log_artifact(local_path="models/lin_reg.bin", artifact_path="models_pickle")

In [42]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [43]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [44]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

[22:02:41] WARNING: ../src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:1247.62268                        
[1]	validation-rmse:1248.07171                        
[2]	validation-rmse:1269.88145                        
[3]	validation-rmse:1266.85977                        
[4]	validation-rmse:1265.33324                        
[5]	validation-rmse:1284.61968                        
[6]	validation-rmse:1298.22617                        
[7]	validation-rmse:1294.13967                        
[8]	validation-rmse:1299.40315                        
[9]	validation-rmse:1311.91949                        
[10]	validation-rmse:1332.47588                       
[11]	validation-rmse:1334.99222                       
[12]	validation-rmse:1337.40995                       
[13]	validation-rmse:1349.75035                       
[14]	validation-rmse:1353.64366                       
[15]	validation-rmse:1353.48797                       
[

In [46]:
mlflow.sklearn.autolog()

for model_class in (RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, LinearSVR):

    with mlflow.start_run():

        mlflow.log_param("data-path", "./data/NYC-Airbnb-2023.csv")

        

        mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

        mlmodel = model_class()
        mlmodel.fit(X_train, y_train)

        y_pred = mlmodel.predict(X_val)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)
        

2023/07/24 22:12:30 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/ubuntu/.local/lib/python3.10/site-packages/mlflow/utils/autologging_utils/safety.py:532: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel()."
2023/07/24 22:13:48 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/ubuntu/.local/lib/python3.10/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils."
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_gb.py:437: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
2023/07/24 22:13:52 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/ubuntu/.local/lib/python3.10/site-packages/mlf